This will be my file to create a plotly dashboard

In [2]:
import dash
from dash import html, dcc
from dash.dependencies import Input, Output, ClientsideFunction

import numpy as np
import pandas as pd
import plotly.express as px
import datetime
from datetime import datetime as dt
import pathlib

In [3]:
app = dash.Dash(
    __name__,
    meta_tags=[{"name": "viewport", "content": "width=device-width, initial-scale=1"}],
)
app.title = "ChatGPT App Review Dashboard"

server = app.server
app.config.suppress_callback_exceptions = True


In [4]:
# Read data
df = pd.read_csv("../data/ChatGPT-play-reviews_sentiment.csv", parse_dates=['at'])
df['Reviews'] = 1
df['at'] = df['at'].dt.date

score_list = df["score"].unique().tolist()
version_list = df["appVersion"].sort_values(ascending=True).unique().tolist()

topics = ["feature evaluation", "praise", "bug report", "feature request", "performance", "usage"]
df["topic"] = np.random.choice(topics, size=len(df), p=[0.2, 0.2, 0.2, 0.2, 0.1, 0.1])
topic_list = df["topic"].unique().tolist()

In [20]:
# Date
# # Format checkin Time
# df["at"] = df["at"].apply(
#     lambda x: dt.strptime(x, "%Y-%m-%d %I:%M:%S %p")
# )  # String -> Datetime

# # Insert weekday and hour of checkin time
# df["at_wd"] = df["Check-In Hour"] = df["at"]
# df["at_wd"] = df["at_wd"].apply(
#     lambda x: dt.strftime(x, "%A")
# )  # Datetime -> weekday string

# df["Review Hour"] = df["at"].apply(
#     lambda x: dt.strftime(x, "%I %p")
# )  # Datetime -> int(hour) + AM/PM

# check_in_duration = df["Check-In Time"].describe()

# # Register all departments for callbacks
# all_departments = df["Department"].unique().tolist()
# wait_time_inputs = [
#     Input((i + "_wait_time_graph"), "selectedData") for i in all_departments
# ]
# score_inputs = [Input((i + "_score_graph"), "selectedData") for i in all_departments]


def description_card():
    """

    :return: A Div containing dashboard title & descriptions.
    """
    return html.Div(
        id="description-card",
        children=[
            html.H5("Market Research on AI Apps"),
            html.H3("Welcome to the AInalyzer"),
            html.Div(
                id="intro",
                children="Explore satisfaction and needs for the gold standard on the current market for AI Apps: The ChatGPT App.",
            ),
        ],
    )


def generate_control_card():
    """

    :return: A Div containing controls for graphs.
    """
    return html.Div(
        id="control-card",
        children=[
            html.P("Select App Version"),
            dcc.Dropdown(
                id="version-select",
                options=[{"label": i, "value": i} for i in score_list],
                value=version_list[0],
            ),
            html.Br(),
            html.P("Select Review Date"),
            dcc.DatePickerRange(
                id="date-picker-select",
                start_date=pd.to_datetime(df['at']).min(), # dt(2014, 1, 1),
                end_date=pd.to_datetime(df['at']).max(), #dt(2014, 1, 15),
                min_date_allowed=pd.to_datetime(df['at']).min(),
                max_date_allowed=pd.to_datetime(df['at']).max(),
                initial_visible_month=dt(2023, 10, 1),
            ),
            html.Br(),
            html.Br(),
            html.P("Select Review Topic"),
            dcc.Dropdown(
                id="topic-select",
                options=[{"label": i, "value": i} for i in topic_list],
                value=topic_list[:],
                multi=True,
            ),
            html.Br(),
            html.Div(
                id="reset-btn-outer",
                children=html.Button(id="reset-btn", children="Reset", n_clicks=0),
            ),
        ],
    )


def generate_patient_volume_heatmap(start, end, version_list, topic_list, reset):
    """
    :param: start: start date from selection.
    :param: end: end date from selection.
    :param: version_list: version_list from selection.
    :param: hm_click: clickData from heatmap.
    :param: topic_list: admission type from selection.
    :param: reset (boolean): reset heatmap graph if True.

    :return: Patient volume annotated heatmap.
    """

    filtered_df = df[
        (df["appVersion"] == version_list) & (df["topic"].isin(topic_list))
    ]
    filtered_df = filtered_df.sort_values("at").set_index("at")[
        start:end
    ]

    x_axis = [datetime.time(i).strftime("%I %p") for i in range(24)]  # 24hr time list
    y_axis = topic_list

    hour_of_day = ""
    weekday = ""
    shapes = []

    # Get z value : sum(number of records) based on x, y,

    z = np.zeros((7, 24))
    annotations = []

    for ind_y, day in enumerate(y_axis):
        filtered_day = filtered_df[filtered_df["at_wd"] == day]
        for ind_x, x_val in enumerate(x_axis):
            sum_of_record = filtered_day[filtered_day["Check-In Hour"] == x_val][
                "Number of Records"
            ].sum()
            z[ind_y][ind_x] = sum_of_record

            annotation_dict = dict(
                showarrow=False,
                text="<b>" + str(sum_of_record) + "<b>",
                xref="x",
                yref="y",
                x=x_val,
                y=day,
                font=dict(family="sans-serif"),
            )
            # Highlight annotation text by self-click
            if x_val == hour_of_day and day == weekday:
                if not reset:
                    annotation_dict.update(size=15, font=dict(color="#ff6347"))

            annotations.append(annotation_dict)

    # Heatmap
    hovertemplate = "<b> %{y}  %{x} <br><br> %{z} Patient Records"

    data = [
        dict(
            x=x_axis,
            y=y_axis,
            z=z,
            type="heatmap",
            name="",
            hovertemplate=hovertemplate,
            showscale=False,
            colorscale=[[0, "#caf3ff"], [1, "#2c82ff"]],
        )
    ]

    layout = dict(
        margin=dict(l=70, b=50, t=50, r=50),
        modebar={"orientation": "v"},
        font=dict(family="Open Sans"),
        annotations=annotations,
        shapes=shapes,
        xaxis=dict(
            side="top",
            ticks="",
            ticklen=2,
            tickfont=dict(family="sans-serif"),
            tickcolor="#ffffff",
        ),
        yaxis=dict(
            side="left", ticks="", tickfont=dict(family="sans-serif"), ticksuffix=" "
        ),
        hovermode="closest",
        showlegend=False,
    )
    return {"data": data, "layout": layout}

app.layout = html.Div(
    id="app-container",
    children=[
        # Banner
        html.Div(
            id="banner",
            className="banner",
            children=[html.Img(src=app.get_asset_url("https://upload.wikimedia.org/wikipedia/commons/0/04/ChatGPT_logo.svg"))],
        ),
        # Left column
        html.Div(
            id="left-column",
            className="four columns",
            children=[description_card(), generate_control_card()]
            + [
                html.Div(
                    ["initial child"], id="output-clientside", style={"display": "none"}
                )
            ],
        ),
        # Right column
        html.Div(
            id="right-column",
            className="eight columns",
            children=[
                # Chart 1
                html.Div(
                    id="patient_volume_card",
                    children=[
                        html.B("Patient Volume"),
                        html.Hr(),
                        dcc.Graph(id="patient_volume_hm"),
                    ],
                ),
                # Chart 2
                html.Div(
                    id="wait_time_card",
                    children=[
                        html.B("Patient Wait Time and Satisfactory Scores"),
                        html.Hr(),
                        html.Div(id="wait_time_table", children=initialize_table()),
                    ],
                ),
            ],
        ),
    ],
)


@app.callback(
    #Output("patient_volume_hm", "figure"),
    [
        Input("date-picker-select", "start_date"),
        Input("date-picker-select", "end_date"),
        Input("version-select", "value"),
        #Input("patient_volume_hm", "clickData"),
        Input("topic-select", "value"),
        Input("reset-btn", "n_clicks"),
    ],
)
# def update_heatmap(start, end, version_list, topic_list, reset_click):
#     start = start + " 00:00:00"
#     end = end + " 00:00:00"

#     reset = False
#     # Find which one has been triggered
#     ctx = dash.callback_context

#     if ctx.triggered:
#         prop_id = ctx.triggered[0]["prop_id"].split(".")[0]
#         if prop_id == "reset-btn":
#             reset = True

#     # Return to original hm(no colored annotation) by resetting
#     return generate_patient_volume_heatmap(
#         start, end, version_list, hm_click, topic_list, reset
#     )




@app.callback(
    #Output("wait_time_table", "children"),
    [
        Input("date-picker-select", "start_date"),
        Input("date-picker-select", "end_date"),
        Input("version-select", "value"),
        Input("topic-select", "value"),
        Input("reset-btn", "n_clicks"),
    ]
)
def update_table(start, end, version_list, topic_list, reset_click, *args):
    start = start + " 00:00:00"
    end = end + " 00:00:00"

    # Find which one has been triggered
    ctx = dash.callback_context

    prop_id = ""
    prop_type = ""
    triggered_value = None
    if ctx.triggered:
        prop_id = ctx.triggered[0]["prop_id"].split(".")[0]
        prop_type = ctx.triggered[0]["prop_id"].split(".")[1]
        triggered_value = ctx.triggered[0]["value"]

    # filter data
    filtered_df = df[
        (df["appVersion"] == version_list) & (df["topic"].isin(topic_list))
    ]
    filtered_df = filtered_df.sort_values("at").set_index("at")[
        start:end
    ]


# Run the server
if __name__ == "__main__":
    app.run_server(debug=True)

NameError: name 'initialize_table' is not defined

In [ ]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)


app.layout = html.Div([
    html.H4(children='ChatGPT - AI Market Insights', style={'textAlign':'center'}),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=[0, 1, 2, 3, 4, 5],
        value=[0, 1, 2, 3, 4, 5],
        inline=True,
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(scores):
    mask = df.score.isin(scores)
    title = 'Number of Reviews'
    fig = px.line(df[mask], x='at', y='Reviews', title=title, color='score')
    fig.update_layout(xaxis_title='Month',
                   yaxis_title='Reviews')
    return fig

app.run_server(debug=True)

In [15]:
app = Dash(__name__)

app.layout = html.Div([
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.topic.unique(), 'Canada', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
])

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.topic==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run(debug=True)

AttributeError: 'DataFrame' object has no attribute 'country'

In [9]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

df_long_plus['at']).dt.date.


app.layout = html.Div([
    html.H4(children='ChatGPT - AI Market Insights', style={'textAlign':'center'}),
    dcc.Graph(id="graph"),
    dcc.Checklist(
        id="checklist",
        options=[0, 1, 2, 3, 4, 5],
        value=[0, 1, 2, 3, 4, 5],
        inline=True,
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def update_line_chart(scores):
    mask = df.score.isin(scores)
    title = 'Number of Reviews'
    fig = px.line(df[mask], x='at', y='Reviews', title=title, color='score')
    fig.update_layout(xaxis_title='Month',
                   yaxis_title='Reviews')
    return fig

app.run_server(debug=True)


---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
File ~/Documents/Bootcamp/Git/Capstone/capstone_chat-gpt/.venv/lib/python3.11/site-packages/plotly/express/_chart_types.py:264, in line(
    data_frame=                                 reviewId       ...56  feature evaluation  

[400 rows x 35 columns],
    x='at',
    y='Reviews',
    line_group=None,
    color='score',
    line_dash=None,
    symbol=None,
    hover_name=None,
    hover_data=None,
    custom_data=None,
    text=None,
    facet_row=None,
    facet_col=None,
    facet_col_wrap=0,
    facet_row_spacing=None,
    facet_col_spacing=None,
    error_x=None,
    error_x_minus=None,
    error_y=None,
    error_y_minus=None,
    animation_frame=None,
    animation_group=None,
    category_orders=None,
    labels=None,
    orientation=None,
    color_discrete_sequence=None,
    color_discrete_map=None,
    line_dash_sequence=None

In [50]:
mask = df["score"] == 2
px.histogram(df[mask], 
       y=df[mask].topic, 
       x=df[mask].Reviews,
       orientation='h',
       histnorm = 'percent')

In [36]:
df_chart[mask].groupby('at_ym', as_index=False).sum('Reviews')

/var/folders/vl/wpx53sjn0nv5cmny0jckrfjr0000gn/T/ipykernel_24189/2927401002.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,at_ym,Reviews
0,2023-07,1152
1,2023-08,739
2,2023-09,1062
3,2023-10,1112


In [8]:
mask = df["score"] == 4
fig = px.line(df[mask].groupby('at_ym', as_index=False).sum('Reviews'), 
       y='Reviews', 
       x='at_ym',
       title="Monthly Number of Reviews"
       text='Reviews',
       labels={'at_ym': 'Review Date'},
       template='ggplot2',
       hover_data={"at_ym": "|%B %d, %Y"},)
fig.update_xaxes(
    dtick="M1",
    tickformat="%B\n%Y")
fig.show()

In [15]:
mask = df["score"] == 4
fig = px.line(df[mask].groupby(['at', 'appVersion'], as_index=False).sum('Reviews'), 
       y='Reviews', 
       x='at',
       title="Daily Number of Reviews by AppVersion"
       labels={'at_ym': 'Review Date'},
       template='ggplot2',
       hover_data={"at": "|%B %d, %Y"},
       color='appVersion')
fig.update_xaxes(
    dtick="M1",
    tickformat="%d.%B\n%Y")
fig.show()

In [43]:
mask = df["score"] == 4
px.line(df[mask].groupby(['at', 'appVersion'], as_index=False).sum('Reviews'), 
       y='Reviews', 
       x='at',
       orientation='h',
       color='appVersion')

In [1]:
from dash import Dash, dcc, html, Input, Output
import plotly.express as px

app = Dash(__name__)

def description_card():
    """

    :return: A Div containing dashboard title & descriptions.
    """
    return html.Div(
        id="description-card",
        children=[
            html.H5("Market Research on AI Apps"),
            html.H3("Welcome to the AInalyzer"),
            html.Div(
                id="intro",
                children="Explore satisfaction and needs for the gold standard on the current market for AI Apps: The ChatGPT App.",
            ),
        ],
    )


def generate_control_card():
    """

    :return: A Div containing controls for graphs.
    """
    return html.Div(
        id="control-card",
        children=[
            html.P("Select App Version"),
            dcc.Dropdown(
                id="version-select",
                options=[{"label": i, "value": i} for i in version_list],
                value=version_list,
            ),
            html.Br(),
            html.P("Select Review Date"),
            dcc.DatePickerRange(
                id="date-picker-select",
                start_date=pd.to_datetime(df['at']).min(), # dt(2014, 1, 1),
                end_date=pd.to_datetime(df['at']).max(), #dt(2014, 1, 15),
                min_date_allowed=pd.to_datetime(df['at']).min(),
                max_date_allowed=pd.to_datetime(df['at']).max(),
                initial_visible_month=dt(2023, 10, 1),
            ),
            html.Br(),
            html.Br(),
            html.P("Select Review Topic"),
            dcc.Dropdown(
                id="topic-select",
                options=[{"label": i, "value": i} for i in topic_list],
                value=topic_list[:],
                multi=True,
            ),
            html.Br(),
            html.Div(
                id="reset-btn-outer",
                children=html.Button(id="reset-btn", children="Reset", n_clicks=0),
            ),
        ],
    )



app.layout = html.Div([
    html.H4('Life expentancy progression of countries per continents'),
    dcc.Graph(id="graph"),
    html.P("Select stock:"),
    dcc.Checklist(
        id="checklist",
        options=version_list,
        value=version_list,
        inline=True
    ),
])


@app.callback(
    Output("graph", "figure"), 
    Input("checklist", "value"))
def display_line_chart(start, end, version_list, topic_list, reset):
    filtered_df = df[
        (df["appVersion"] == version_list) & (df["topic"].isin(topic_list))
    ]
    filtered_df = filtered_df.sort_values("at").set_index("at")[
        start:end
    ]
    fig = px.line(filtered_df.groupby(['at', 'score'], as_index=False).sum('Reviews'), 
        y='Reviews', 
        x='at',
        title="Daily Number of Reviews by AppVersion",
        labels={'at_ym': 'Review Date'},
        template='ggplot2',
        hover_data={"at": "|%B %d, %Y"},
        color='score')
    fig.update_xaxes(
        dtick="M1",
        tickformat="%d.%B\n%Y")
    return fig
    
app.layout = html.Div(
    id="app-container",
    children=[
        # Banner
        html.Div(
            id="banner",
            className="banner",
            children=[html.Img(src=app.get_asset_url("https://upload.wikimedia.org/wikipedia/commons/0/04/ChatGPT_logo.svg"))],
        ),
        # Left column
        html.Div(
            id="left-column",
            className="four columns",
            children=[description_card(), generate_control_card()]
            + [
                html.Div(
                    ["initial child"], id="output-clientside", style={"display": "none"}
                )
            ],
        ),
        # Right column
        html.Div(
            id="right-column",
            className="eight columns",
            children=[
                # Chart 1
                html.Div(
                    id="line_graph",
                    children=[
                        html.B("Daily Number of Reviews"),
                        html.Hr(),
                        dcc.Graph(id="line_graph"),
                    ],
                ),
                # # Chart 2
                # html.Div(
                #     id="wait_time_card",
                #     children=[
                #         html.B("Patient Wait Time and Satisfactory Scores"),
                #         html.Hr(),
                #         html.Div(id="wait_time_table", children=initialize_table()),
                #     ],
                # ),
            ],
        ),
    ],
)


@app.callback(
    Output("graph", "figure"),
    [
        Input("date-picker-select", "start_date"),
        Input("date-picker-select", "end_date"),
        Input("version-select", "value"),
        Input("topic-select", "value"),
        Input("reset-btn", "n_clicks"),
    ],
)


# Run the server
if __name__ == "__main__":
    app.run_server(debug=True)


SyntaxError: invalid syntax (3944461912.py, line 165)